<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --quiet spacy
!pip install "spacy-pkuseg>=0.0.27,<0.1.0"
!python -m spacy download zh_core_web_lg
# !wget https://github.com/explosion/spacy-models/releases/download/zh_core_web_md-3.1.0/zh_core_web_md-3.1.0.tar.gz
# !tar xvfz zh_core_web_md-3.1.0.tar.gz
# !python -m spacy download zh_core_web_trf

     |████████████████████████████████| 6.4 MB 6.5 MB/s 
     |████████████████████████████████| 10.1 MB 48.2 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 456 kB 52.3 MB/s 
     |████████████████████████████████| 621 kB 49.3 MB/s 
     |████████████████████████████████| 2.4 MB 7.7 MB/s 
2021-08-10 22:39:40.026752: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 603.8 MB 8.6 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [17]:
import spacy
from spacy import displacy

# spacyModel='zh_core_web_md-3.1.0/zh_core_web_md/zh_core_web_md-3.1.0'
nlp = spacy.load('zh_core_web_lg')

doc = nlp('恆指漲3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。')
displacy.render(doc,jupyter=True,style="ent")

# for ent in doc.ents:
#   print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [18]:
import random
from spacy.training.example import Example

TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN')         
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),(17,19,'STOCK')      
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '匯控低開4%。港股收市跌。',
    {
        'entities':
        [
         (0,2,'FIN'),(7,9,'STOCK')
        ]
    }
  ),
  (
    '港股高開173點。',
   {
       'entities': [(0,2,'FIN'),(2,4,'FIN')]
   }
  )
]

ner = nlp.get_pipe('ner')

ner.add_label('STOCK')
ner.add_label('FIN')

epoch = 50
optimizer = nlp.resume_training()

# get names of other pipes to disable them during training
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print(f'Pipes: {other_pipes}')
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = spacy.util.minibatch(TRAIN_DATA,size=2)
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.1,sgd=optimizer,losses=losses)
    print("Losses", losses)

nlp.to_disk('saved_model')

print(f'NER labels: {ner.labels}')

Pipes: ['tok2vec', 'tagger', 'parser', 'attribute_ruler']
Losses {'ner': 52.35395499326036}
Losses {'ner': 35.31875653182033}
Losses {'ner': 30.13936422394454}
Losses {'ner': 24.693429656555963}
Losses {'ner': 22.369038288212323}
Losses {'ner': 18.15978593742102}
Losses {'ner': 15.218005186814707}
Losses {'ner': 13.730889743091208}
Losses {'ner': 12.829818910772607}
Losses {'ner': 27.667409023842907}
Losses {'ner': 34.67827834343304}
Losses {'ner': 16.444100633860693}
Losses {'ner': 11.322313622207702}
Losses {'ner': 8.582245246464359}
Losses {'ner': 8.44532377990693}
Losses {'ner': 9.491919096082805}
Losses {'ner': 7.02237693556344}
Losses {'ner': 3.1088403212996205}
Losses {'ner': 0.9621571045818466}
Losses {'ner': 0.34195758238378127}
Losses {'ner': 0.041072559916830176}
Losses {'ner': 0.005893571473067863}
Losses {'ner': 0.020797606180810193}
Losses {'ner': 0.0024096335793814274}
Losses {'ner': 2.365749159034766e-05}
Losses {'ner': 1.205935432210743e-05}
Losses {'ner': 0.0008032879

In [4]:
# for text, annot in TRAIN_DATA:
#     print(text,annot)
#     doc = nlp.make_doc(text)
#     biluo = spacy.training.offsets_to_biluo_tags(doc, annot["entities"])
#     print([t.text for t in doc], biluo)

In [19]:
# Let's give our trained model a try!

texts = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

nlp_updated = spacy.load('saved_model')

for text in texts:
  doc = nlp_updated(text)
  displacy.render(doc,jupyter=True,style="ent")
